**Stock Price Prediction using ARIMA model by Aana Kakroo**

Import the required Python libraries

In [ ]:
#importing python libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

Import datasets into Jupyter Notebook

In [ ]:
#importing python module glob which is used to return all file paths that match a specific pattern
import glob

#path of the data file in the github repository clone on my local system
path = r'C:\Users\aanak\OneDrive\Desktop\Materials for college\DAC\NIFTY-Stock-Market-Prediction\data'

#using glob to get all csv files from the folder given in the path
all_files = glob.glob(path + "/*csv")
all_files

#concatenating all csv files and storing the data as a single pandas dataframe called "df"
df = pd.concat((pd.read_csv(f) for f in all_files))
df


Checking number of rows and columns in the dataframe

In [ ]:
df.shape

Getting statistical information about the data in the dataframe

In [ ]:
df.describe()

Checking the datatypes of the values in each column of the dataframe

In [ ]:
df.info()

In [ ]:
df.drop(['ISIN Code','Company Name','Industry'],axis=1,inplace=True)
df.head()

The "Date" column is an object datatype and needs to be changed to datetime datatype

In [ ]:
# converting the string to datetime format
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.info()

Checking number of NULL values in the dataset

In [ ]:
df.isna().sum()

As observed from above 'Date', 'Prev Close', 'Open', 'High', 'Low', 'Last, 'Close', 'VWAP', 'Volume', 'Turnover' columns have same amount of NULL values, that is 50 NULL values. It is possible that the 50 NULL values in the above columns belong to the same 50 rows, rather than being randomly scattered across the dataframe.

In [ ]:
#displaying the rows which have NULL values in the 'Date' column
df[df['Date'].isna()]

The above rows can be dropped as crucial information is missing or NULL.

In [ ]:
#dropping rows in df which have NULL values in the 'Date' column
df.drop(df[df['Date'].isna()].index, inplace = True)
df.isna().sum()

In [ ]:
#displaying the rows which have NULL values in the 'Trades' column
df[df['Trades'].isna()]

Displaying the different Symbols of shares in the NIFTY 50 from the dataframe

In [ ]:
df['Symbol'].unique()

In [ ]:
df = pd.concat([df[].shift(3), temps.shift(2), temps.shift(1), temps], axis=1)
df.columns = ['t-2', 't-1', 't', 't+1']